In [1]:
import pandas as pd
import defs
import requests

In [2]:
sessions = requests.Session()

In [3]:
instrument_df = pd.read_pickle('instruments.pkl')

In [4]:
currencies = ['EUR', 'USD', 'AUD', 'GBP', 'NZD', 'CAD']

In [6]:
def fetch_candles(pair_name, count, granularity):
    url = f'{defs.OANDA_URL}/instruments/{pair_name}/candles'
    params = dict(
        count = count,
        granularity = granularity,
        price = 'MBA'
    )
    response = sessions.get(url, params=params, headers=defs.SECURE_HEADER)
    return response.status_code, response.json()

In [7]:
code, res = fetch_candles('EUR_USD', 10, 'H1')

In [8]:
def get_candles_df(json_response):
    prices = ['mid', 'bid', 'ask']
    ohlc = ['o', 'h', 'l', 'c']

    my_data = []
    for candle in json_response['candles']:
        if candle['complete'] == False:
            continue
        new_dict = {}
        new_dict['time'] = candle['time']
        new_dict['volume'] = candle['volume']        
        for price in prices:
            for oh in ohlc:
                new_dict[f'{price}_{oh}'] = candle[price][oh]
        my_data.append(new_dict)
    return pd.DataFrame.from_dict(my_data)


In [9]:
df = get_candles_df(res)

In [11]:
def save_file(candles_df, pair, granularity):
    candles_df.to_pickle(f'historical_data/{pair}_{granularity}.pkl')

In [22]:
def create_data(pair, granularity):
    code, json_data = fetch_candles(pair, 4000, granularity)
    if code != 200:
        print(pair, code, 'Error')
        return
    df = get_candles_df(json_data)
    print(f'{pair} loaded {df.shape[0]} candles from {df.time.min()} to {df.time.max()}')
    save_file(df, pair, granularity)

In [20]:
create_data('EUR_USD', 'H1')

EUR_USD loaded 9 candles from 2024-02-13T11:00:00.000000000Z to 2024-02-13T19:00:00.000000000Z


In [23]:
for pair1 in currencies:
    for pair2 in currencies:            
        pair = f'{pair1}_{pair2}'
        if pair in instrument_df.name.unique():
            create_data(pair, 'H1')
    

EUR_USD loaded 3999 candles from 2023-06-22T04:00:00.000000000Z to 2024-02-13T19:00:00.000000000Z
EUR_AUD loaded 3999 candles from 2023-06-22T04:00:00.000000000Z to 2024-02-13T19:00:00.000000000Z
EUR_GBP loaded 3999 candles from 2023-06-22T04:00:00.000000000Z to 2024-02-13T19:00:00.000000000Z
EUR_NZD loaded 3999 candles from 2023-06-22T05:00:00.000000000Z to 2024-02-13T19:00:00.000000000Z
EUR_CAD loaded 3999 candles from 2023-06-22T04:00:00.000000000Z to 2024-02-13T19:00:00.000000000Z
USD_CAD loaded 3999 candles from 2023-06-22T04:00:00.000000000Z to 2024-02-13T19:00:00.000000000Z
AUD_USD loaded 3999 candles from 2023-06-22T04:00:00.000000000Z to 2024-02-13T19:00:00.000000000Z
AUD_NZD loaded 3999 candles from 2023-06-22T06:00:00.000000000Z to 2024-02-13T19:00:00.000000000Z
AUD_CAD loaded 3999 candles from 2023-06-22T04:00:00.000000000Z to 2024-02-13T19:00:00.000000000Z
GBP_USD loaded 3999 candles from 2023-06-22T04:00:00.000000000Z to 2024-02-13T19:00:00.000000000Z
GBP_AUD loaded 3999 